In [1]:
import pandas as pd
import random
import torch
import flash

from sklearn.model_selection import train_test_split
from flash import Trainer
from flash.audio import SpeechRecognitionData, SpeechRecognition

/home/sd/works/practice-torch/.venv/lib/python3.10/site-packages/pl_bolts/callbacks/data_monitor.py:20: UnderReviewWarning: The feature warn_missing_pkg is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  warn_missing_pkg("wandb")
/home/sd/works/practice-torch/.venv/lib/python3.10/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:35: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/home/sd/works/pra

In [2]:
print("pandas version:",pd.__version__)
print("torch version:",torch.__version__)
print("flash version:",flash.__version__)

pandas version: 1.5.1
torch version: 1.13.0+cu116
flash version: 0.8.1


In [3]:
random.seed(10)
df_scottish = pd.read_csv("./scottish_english_female/line_index.csv", header=None, names=['not_required', 'speech_files', 'targets'])
df_scottish = df_scottish.sample(frac=0.06)
print(df_scottish.shape)
df_scottish.head()

(54, 3)


,not_required,speech_files,targets
620,EN0049,scf_07049_00239785467,The powers of the appointed chair are limited...
585,EN0013,scf_05223_01383412777,Aristotle thought that the rainbow was caused...
597,EN1082,scf_02484_01672323659,In some locales the standard paper-foil packs...
215,EN1210,scf_03397_00052556130,Regarding another work's assumed connection a...
386,SC0034,scf_06136_00400788562,It is eleven degrees and mostly sunny in Pete...


In [4]:
df_scottish = df_scottish[['speech_files', 'targets']]
df_scottish['speech_files'] = './scottish_english_female/' + df_scottish['speech_files'].str.lstrip()
df_scottish['speech_files'] = df_scottish['speech_files'].astype(str) + '.wav'
df_scottish.head()

,speech_files,targets
620,./scottish_english_female/scf_07049_0023978546...,The powers of the appointed chair are limited...
585,./scottish_english_female/scf_05223_0138341277...,Aristotle thought that the rainbow was caused...
597,./scottish_english_female/scf_02484_0167232365...,In some locales the standard paper-foil packs...
215,./scottish_english_female/scf_03397_0005255613...,Regarding another work's assumed connection a...
386,./scottish_english_female/scf_06136_0040078856...,It is eleven degrees and mostly sunny in Pete...


In [5]:
random.seed(42)
train_scottish, test_scottish_raw = train_test_split(df_scottish, test_size=0.2)
test_scottish = test_scottish_raw['speech_files']
test_scottish.head()

669    ./scottish_english_female/scf_03397_0110873286...
439    ./scottish_english_female/scf_05223_0013792456...
621    ./scottish_english_female/scf_03397_0155538275...
518    ./scottish_english_female/scf_04310_0142796824...
317    ./scottish_english_female/scf_02484_0082687193...
Name: speech_files, dtype: object

In [6]:
train_scottish.to_csv('train_scottish.csv')
test_scottish.to_csv('test_scottish.csv')

In [7]:
datamodule = SpeechRecognitionData.from_csv(
     "speech_files",
     "targets",
     train_file="train_scottish.csv",
     predict_file="test_scottish.csv",
     batch_size=4)

/home/sd/works/practice-torch/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3433: FutureWarning: Please pass an instantiated object of the `InputTransform` class. Passing the Class and keyword arguments separately has been deprecated since v0.8.0 and will be removed in v0.9.0.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
SpeechRecognition.available_backbones()

['Anything available from: Hugging Face/transformers (https://github.com/huggingface/transformers)',
 'facebook/wav2vec2-base-960h',
 'facebook/wav2vec2-large-960h-lv60']

In [9]:
model = SpeechRecognition(backbone="facebook/wav2vec2-base-960h")

Using 'facebook/wav2vec2-base-960h' provided by Hugging Face/transformers (https://github.com/huggingface/transformers) and PyTorch/fairseq (https://github.com/pytorch/fairseq).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
print(model.get_backbone_details("facebook/wav2vec2-base-960h"))

[('model_args', <Parameter "*model_args">), ('kwargs', <Parameter "**kwargs">)]


In [11]:
trainer = Trainer(max_epochs=4, gpus=-1, precision=16)
trainer.finetune(model, datamodule=datamodule, strategy="no_freeze")

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type           | Params
-------------------------------------------------
0 | model         | Wav2Vec2ForCTC | 94.4 M
1 | train_metrics | ModuleDict     | 0     
2 | val_metrics   | ModuleDict     | 0     
3 | test_metrics  | ModuleDict     | 0     
-------------------------------------------------
94.4 M    Trainable params
0         Non-trainable params
94.4 M    Total params
188.793   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=4` reached.


In [17]:
trainer.predict(model, datamodule=datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 10it [00:00, ?it/s]

[['<unk>']]

In [20]:
train_scottish['speech_files'][0]

'./scottish_english_female/scf_04310_01356369773.wav'

In [21]:
datamodule = SpeechRecognitionData.from_files(predict_files='./scottish_english_female/scf_02484_00097285334.wav', batch_size=10)
predictions = trainer.predict(model, datamodule=datamodule)
print(predictions)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 10it [00:00, ?it/s]

[['<unk>']]


In [15]:
test_scottish_raw.head()

,speech_files,targets
669,./scottish_english_female/scf_03397_0110873286...,It is sixteen degrees with a chance of storms...
439,./scottish_english_female/scf_05223_0013792456...,In 2015 Disney achieved its largest yearly bo...
621,./scottish_english_female/scf_03397_0155538275...,Many complicated ideas about the rainbow have...
518,./scottish_english_female/scf_04310_0142796824...,The crown is adorned with diamonds totalling ...
317,./scottish_english_female/scf_02484_0082687193...,Alice was default reading to the point where ...


In [16]:
datamodule = SpeechRecognitionData.from_files(predict_files='./scottish_english_female/scf_02484_01925002707.wav', batch_size=10) # Scottish

predictions = trainer.predict(model, datamodule=datamodule)
print(predictions)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 10it [00:00, ?it/s]

[['<unk>']]
